In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.5 MB/s eta 0:00:00


In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00


In [3]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00


In [4]:
pip install accelerate -U

In [5]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.5 MB/s eta 0:00:00


In [6]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding

tqdm.pandas()

dataset = load_dataset("AmazonScience/massive")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
checkpoint = "cartesinus/xlm-r-base-amazon-massive-slot"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["utt"], truncation=True)

In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/587214 [00:00<?, ? examples/s]

Map:   0%|          | 0/103683 [00:00<?, ? examples/s]

Map:   0%|          | 0/151674 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
tokenized_datasets['train'][0].keys()

dict_keys(['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments', 'input_ids', 'attention_mask'])

In [11]:
to_remove = list(tokenized_datasets['train'][0].keys())

to_remove.remove('input_ids')
to_remove.remove('attention_mask')
to_remove.remove('intent')

tokenized_datasets = tokenized_datasets.remove_columns(to_remove)
tokenized_datasets = tokenized_datasets.rename_column('intent', 'labels')

In [12]:
tokenized_datasets['train'].features

{'labels': ClassLabel(names=['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock', 'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe', 'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up', 'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup', 'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query', 'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query', 'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim', 'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down', 'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff', 'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music', 'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set', 'play_game', 'alarm_remove', 'lists_remove', 'transpor

In [13]:
tokenized_datasets['train'][0]

{'labels': 48,
 'input_ids': [0,
  100383,
  677,
  51367,
  218,
  27842,
  115481,
  161936,
  135,
  114759,
  11404,
  140225,
  2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
label_id = tokenized_datasets['train'][0]['labels']
label_name = tokenized_datasets['train'].features['labels'].names[label_id]
print(label_name)

alarm_set


In [15]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [16]:
for batch in tqdm(train_dataloader):
    break
{k: v.shape for k, v in batch.items()}

  0%|          | 0/73402 [00:00<?, ?it/s]


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 31]),
 'attention_mask': torch.Size([8, 31])}

In [17]:
batch['labels']

tensor([54, 19, 11, 45,  0, 45, 55,  0])

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=tokenized_datasets['train'].features['labels'].num_classes)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cartesinus/xlm-r-base-amazon-massive-slot and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [20]:
import numpy as np
import evaluate

metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric_acc.compute(predictions=predictions, references=labels)

In [21]:
train_data_subset = tokenized_datasets["train"].select(range(11500))
eval_data_subset = tokenized_datasets["validation"].select(range(2030))

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon = 1e-08,
    disable_tqdm = False,
    do_eval = True,
    evaluation_strategy = "epoch",
    learning_rate = 2e-05,
    lr_scheduler_type = "linear",
    num_train_epochs = 20,
    optim = "adamw_hf",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    save_strategy = "steps",
    seed = 42,
    weight_decay = 0.01,
    warmup_steps = 0,
    report_to = "none",
    save_steps = 719,
)

In [23]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_data_subset,
    eval_dataset=eval_data_subset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [24]:
stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,2.642700,1.274379,0.709360
2,1.363700,0.833676,0.798030
3,0.692700,0.686638,0.841379
4,0.563000,0.692872,0.838916
5,0.402400,0.671682,0.847291
6,0.329300,0.724725,0.845813
7,0.258700,0.762963,0.850739
8,0.207600,0.784197,0.847291
9,0.172600,0.803409,0.855172
10,0.135600,0.872281,0.857143


In [25]:
from pprint import pprint
pprint(trainer.evaluate())

{'epoch': 20.0,
 'eval_accuracy': 0.8630541871921182,
 'eval_loss': 1.0116207599639893,
 'eval_runtime': 2.7854,
 'eval_samples_per_second': 728.812,
 'eval_steps_per_second': 45.596}


In [34]:
test_data_subset = tokenized_datasets["test"]
preds = trainer.predict(test_data_subset)

In [35]:
test_data_subset_2 = dataset["test"].to_pandas()
test_data_subset_2

,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
0,0,mn-MN,test,16,48,энэ долоо хоног намайг таван цагт сэрээгээрэй,[date : энэ долоо хоног] намайг [time : таван ...,16,"{'slot': ['time', 'date'], 'method': ['transla...","{'worker_id': ['5', '4', '2'], 'intent_score':..."
1,3,mn-MN,test,10,46,чимээгүй,чимээгүй,16,"{'slot': [], 'method': []}","{'worker_id': ['2', '4', '10'], 'intent_score'..."
2,8,mn-MN,test,8,1,бидэнд ягаан өнгө л хэрэгтэй байна,бидэнд [color_type : ягаан] өнгө л хэрэгтэй байна,5,"{'slot': ['color_type'], 'method': ['translati...","{'worker_id': ['38', '30', '2'], 'intent_score..."
3,14,mn-MN,test,8,41,тэгээд гүн харанхуй боллоо,тэгээд гүн харанхуй боллоо,16,"{'slot': [], 'method': []}","{'worker_id': ['4', '45', '5'], 'intent_score'..."
4,19,mn-MN,test,8,40,олли унтлагын өрөөний гэрлийг унтраагаарай,олли [house_place : унтлагын өрөөний] гэрлийг ...,3,"{'slot': ['house_place'], 'method': ['translat...","{'worker_id': ['45', '4', '5'], 'intent_score'..."
...,...,...,...,...,...,...,...,...,...,...
151669,17151,am-ET,test,7,33,ጓደኛዬ ጫላ አዲስ ወደገዛው ቤቱ ግብዣ ካርድ ልኮልኛል በደብዳቤ መታወቂያ...,[relation : ጓደኛዬ] [person : ጫላ] አዲስ ወደገዛው ቤቱ ግ...,17,"{'slot': ['relation', 'person'], 'method': ['t...","{'worker_id': ['7', '5', '50'], 'intent_score'..."
151670,17156,am-ET,test,7,44,ማንኛውም አዲስ የዮሃንስ ኢሜይል ከኢሜይሎቼ ውስጥ አረጋግጥ,ማንኛውም አዲስ [person : የዮሃንስ] ኢሜይል ከኢሜይሎቼ ውስጥ አረጋግጥ,17,"{'slot': ['person'], 'method': ['localization']}","{'worker_id': ['5', '50', '49'], 'intent_score..."
151671,17157,am-ET,test,7,44,ከዮሐንስ ማንኛውም አዲስ ኢሜይል አለኝ,[person : ከዮሐንስ] ማንኛውም አዲስ ኢሜይል አለኝ,17,"{'slot': ['person'], 'method': ['localization']}","{'worker_id': ['50', '5', '28'], 'intent_score..."
151672,17170,am-ET,test,7,44,ኢያሱ ኢሜይሎች ሲልክልኝ አሳውቀኝ,[person : ኢያሱ] ኢሜይሎች ሲልክልኝ አሳውቀኝ,5,"{'slot': ['person'], 'method': ['localization']}","{'worker_id': ['44', '5', '50'], 'intent_score..."


In [36]:
import numpy as np

label_ids = np.array(preds.predictions.argmax(-1))
label_names = [tokenized_datasets['test'].features['labels'].names[label] for label in label_ids]
print(label_names)

['alarm_set', 'audio_volume_mute', 'iot_hue_lightchange', 'iot_hue_lighton', 'iot_hue_lightoff', 'play_music', 'iot_cleaning', 'general_quirky', 'general_greet', 'datetime_query', 'datetime_convert', 'alarm_remove', 'alarm_query', 'alarm_query', 'music_likeness', 'iot_hue_lightup', 'datetime_query', 'datetime_query', 'takeaway_order', 'weather_query', 'weather_query', 'calendar_query', 'transport_traffic', 'general_joke', 'datetime_query', 'takeaway_order', 'music_likeness', 'play_audiobook', 'iot_hue_lightdim', 'takeaway_query', 'takeaway_query', 'takeaway_query', 'takeaway_query', 'alarm_set', 'news_query', 'news_query', 'play_music', 'play_music', 'audio_volume_mute', 'datetime_query', 'alarm_remove', 'weather_query', 'weather_query', 'play_music', 'play_music', 'play_music', 'audio_volume_up', 'audio_volume_up', 'iot_hue_lightchange', 'weather_query', 'qa_currency', 'iot_hue_lightup', 'iot_cleaning', 'iot_cleaning', 'play_music', 'play_music', 'weather_query', 'datetime_query', 'au

In [37]:
preds.predictions.argmax(-1)

array([48, 46,  1, ..., 44, 44, 33])

In [38]:
print(len(test_data_subset_2['locale']))
print(len(test_data_subset_2['id']))
print(len(label_names))

151674
151674
151674


In [39]:
import pandas as pd

data = {
    "locale" : test_data_subset_2['locale'],
    "id" : test_data_subset_2['id'],
    "intent" : label_names
}

df = pd.DataFrame(data)

In [40]:
df

,locale,id,intent
0,mn-MN,0,alarm_set
1,mn-MN,3,audio_volume_mute
2,mn-MN,8,iot_hue_lightchange
3,mn-MN,14,iot_hue_lighton
4,mn-MN,19,iot_hue_lightoff
...,...,...,...
151669,am-ET,17151,general_quirky
151670,am-ET,17156,email_query
151671,am-ET,17157,email_query
151672,am-ET,17170,email_query


In [42]:
df.to_csv('./result.csv',sep=',',index = False)